In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
# access file
data_file_DZHI = "patient_flow_DZHI.xlsx"

In [ ]:
# load sheets
df_data_dict = pd.read_excel(data_file_DZHI, header=None, sheet_name=None)

In [ ]:
df_data_DZHI = pd.DataFrame()
for date, df in df_data_dict.items():
    df.columns = ["room", "date of birth", "text"]
    df["date of visit"] = pd.to_datetime(date, dayfirst=True).date()
    df["cohort"] = "DZHI"
    df_data_DZHI = pd.concat([df_data_DZHI, df])

In [ ]:
df_data_DZHI = df_data_DZHI.dropna(subset=["date of birth", "room"])

In [ ]:
rooms_to_keep_for_DZHI = []

In [ ]:
df_data_DZHI = df_data_DZHI.loc[df_data_DZHI["room"].isin(rooms_to_keep_for_DZHI), :]

In [ ]:
df_data_DZHI = df_data_DZHI.loc[~df_data_DZHI[["date of birth"]].shift().eq(df_data_DZHI[["date of birth"]]).all(axis=1)]
df_data_DZHI = df_data_DZHI.reset_index(drop=True)

In [ ]:
# access file
data_file_ED = "patient_flow_ED.xlsx"

In [ ]:
# load data
df_data_ED = pd.read_excel(data_file_ED, header=None)
df_data_ED.columns = ["date of visit", "text", "date of birth"]
df_data_ED["date of visit"] = pd.to_datetime(df_data_ED["date of visit"]).dt.date
df_data_ED["cohort"] = "ED"

In [ ]:
df_data_patients_visited = pd.concat([df_data_DZHI[["date of birth", "date of visit", "cohort"]], df_data_ED[["date of birth", "date of visit", "cohort"]]])
df_data_patients_visited["date of visit"] = df_data_patients_visited["date of visit"].astype(str)

In [ ]:
df_patients = pd.DataFrame({"DZHI": df_data_patients_visited.groupby("cohort")["date of visit"].value_counts()["DZHI"], "ED": df_data_patients_visited.groupby("cohort")["date of visit"].value_counts()["ED"]}).fillna(0).astype(int)
df_patients["total"] = df_patients.sum(axis=1).astype(int)

In [ ]:
# access file
data_file = "data.xlsx"

In [ ]:
# load table and transform
df_data_unclean = pd.read_excel(data_file, header=None, index_col=0)
df_data_unclean = df_data_unclean.T
# remove empty entries
df_data_unclean = df_data_unclean.loc[df_data_unclean["pseudonym"].notna(), :]
# replace 'error: missing input'
df_data_unclean = df_data_unclean.replace({'error: missing input': np.nan})

In [ ]:
# check for duplicates
df_data_unclean.loc[df_data_unclean.duplicated(subset=["pseudonym"]), "pseudonym"]

In [ ]:
# remove excluded and unfinished patients via exclusion in excel
print("pseudonyms to remove:", ", ".join(df_data_unclean.loc[df_data_unclean["exclusion due to exclusion criteria"]=="yes", "pseudonym"].astype(str).values))
df_data_unclean_unfinished = df_data_unclean.loc[df_data_unclean["exclusion due to exclusion criteria"]!="yes", :]
print("pseudonyms with unfinished entry:", ", ".join(df_data_unclean_unfinished.loc[df_data_unclean_unfinished["entry done"]=="no", "pseudonym"].astype(str).values))
df_data = df_data_unclean_unfinished.loc[df_data_unclean_unfinished["entry done"]!="no", :]

In [ ]:
cohorts = df_data["cohort"].unique()
# create a per day table and plot it
end_date = "2025-09-12"
patients_per_date_per_cohort = []
patients_per_date_per_cohort.append(df_data.value_counts(subset="date of inclusion").rename("total"))
for cohort in cohorts:
    patients_per_date_per_cohort.append(df_data.loc[df_data["cohort"]==cohort, :].value_counts(subset="date of inclusion").rename(cohort))
df_patients_per_date = pd.concat(patients_per_date_per_cohort, axis=1)
df_patients_per_date = df_patients_per_date.reindex(pd.bdate_range(start="2025-07-01", end=end_date)).fillna(0).astype(int)
df_patients_per_date_reindexed = df_patients_per_date
df_patients_per_date_reindexed.index = df_patients_per_date.index.strftime("%Y-%m-%d (%a)")
df_patients_per_date_percent = df_patients_per_date_reindexed[["DZHI", "ED"]].fillna(0)
df_patients_per_date_long = df_patients_per_date_percent.reset_index().melt(id_vars="index", var_name="cohort", value_name="patients per day")
df_patients_per_date_long = df_patients_per_date_long.rename(columns={"index": "date"})
fig = px.bar(df_patients_per_date_long, x="date", y="patients per day", color="cohort", title="patients per day per cohort", text_auto="")
fig.update_layout(xaxis=dict(type="date", rangebreaks=[dict(bounds=["sat", "mon"])])) # excludes saturday and sunday
fig.update_layout(barmode="stack", xaxis_type="category")
fig.show()
fig.write_image("patient_inclusion_timeline.png", engine="orca", width=2500)
fig.write_image("patient_inclusion_timeline.svg", engine="orca", width=2500)

In [ ]:
# mean patients per day
df_patients_per_date.mean()

In [ ]:
patients_per_date_per_cohort = []
patients_per_date_per_cohort.append(df_data.value_counts(subset="date of inclusion").rename("total"))
for cohort in cohorts:
    patients_per_date_per_cohort.append(df_data.loc[df_data["cohort"]==cohort, :].value_counts(subset="date of inclusion").rename(cohort))
df_patients_included = pd.concat(patients_per_date_per_cohort, axis=1)
df_patients_included = df_patients_included.reindex(pd.date_range(start="2025-07-01", end=end_date).strftime("%Y-%m-%d")).fillna(0).astype(int)
proportion_of_patients_included = (df_patients_included/df_patients)

In [ ]:
df_patients_for_export = df_patients.reindex(pd.date_range(start="2025-07-01", end=end_date).strftime("%Y-%m-%d")).fillna(0).astype(int)
df_patients_for_export.columns=["n DZHI", "n ED", "n total"]

In [ ]:
df_patients_included_for_export=df_patients_included[["DZHI", "ED", "total"]]
df_patients_included_for_export.columns=["n included DZHI", "n included ED", "n included total"]
proportion_of_patients_included_for_export = proportion_of_patients_included
proportion_of_patients_included_for_export.columns=["percentage DZHI", "percentage ED", "percentage total"]
ns_and_percentages_of_included_patients = pd.concat([proportion_of_patients_included_for_export, df_patients_included_for_export, df_patients_for_export], axis = 1)
ns_and_percentages_of_included_patients.to_csv("ns_and_percentages_of_included_patients.csv")

In [ ]:
# ED: impute suspected diagnoses
df_data_ED_with_diagnosis = df_data_ED.dropna().copy(deep=True)
df_data_ED_with_category = df_data_ED.copy(deep=True)
df_data_ED_with_category["category"] = np.nan

In [ ]:
diagnosis_to_category = {    
    "Heart failure": [],
    "Chest pain": [],
    "Dysrhythmia": [], 
    "Pericardial effusion": [],
    "Aortic dissection": [],
    "Hypertension": [],
    "Electrolyte derailment": [],
    "Metabolic derailment": [],
    "Acute infection": [],
    "Pulmonary embolism": [],
    "Other pulmonary causes": [],
    "Dyspnea": [],
    "Abdominal causes": [],
    "Allergic reaction": [],
    "Nephral causes": [],
    "AZ": [],
    "Thrombosis": [],
    "Hematological causes": [],
    "Oncology": [], 
    "Rheumatic causes": [],
    "Psychiatric causes": [],
    "Environmental causes": [],
    "Infectiological causes": [],
    "Neurological causes": [],
    "Hypervolaemia": [],
    "Palliation": [],
    "Other": [],
}

In [ ]:
diagnosis_category_pseudonym = {}
category_count = {k: 0 for k in diagnosis_to_category.keys()}
multiple_categories_n = 0
for i, row in df_data_ED_with_diagnosis.iterrows():
    pseudonym = f'{row["date of visit"]}_{row["date of birth"]}_{row["cohort"]}'
    diagnosis = row["text"]
    category = [c for c, examples in diagnosis_to_category.items() if any([e.replace("-", " ").lower() in diagnosis.replace("-", " ").lower() for e in examples])] 
    diagnosis_category_pseudonym[pseudonym] = {"suspected diagnosis": diagnosis, "category": category, "multiple_categories": len(category)}
    for c in category:
        category_count[c] += 1
    if not category:
        print(diagnosis)
    multiple_categories_n += len(category)
    if "Remove" in category:
        df_data_ED.loc[i, "text"] = np.nan 
        df_data_ED_with_category.loc[i, "text"] = np.nan
    df_data_ED_with_category.loc[i, "category"] = str(category)

In [ ]:
percentage_category_count = {k:v/df_data_ED_with_diagnosis.shape[0] for k, v in category_count.items()}

In [ ]:
pd.Series(percentage_category_count).to_csv("categories_patients_in_ED.csv")

In [ ]:
# ED: number of eligible patients by imputed suspected diagnoses
n_included_categories = 0
n_included_patients_with_category = 0
category_of_included_patients = {k: 0 for k in diagnosis_to_category.keys()}
for i, row in df_data.loc[df_data["cohort"]=="ED", :].iterrows():
    if ((df_data_ED["date of visit"].astype(str) == str(row["day + time at ED visit (for ED cohort only)"].date())) & (df_data_ED["date of birth"]==row["date of birth"])).any():
        text = df_data_ED.loc[(df_data_ED["date of visit"].astype(str) == str(row["day + time at ED visit (for ED cohort only)"].date())) & (df_data_ED["date of birth"]==row["date of birth"]), "text"].values[0]
        categories = df_data_ED_with_category.loc[(df_data_ED["date of visit"].astype(str) == str(row["day + time at ED visit (for ED cohort only)"].date())) & (df_data_ED["date of birth"]==row["date of birth"]), "category"].values[0]
    else:
        print("patient missing in patient flow data", row)
    if not pd.isna(categories):
        for category in categories.split(", "):
            category = category.replace("\'", "")
            category = category.replace("[", "")
            category = category.replace("]", "")
            category_of_included_patients[category] += 1
            n_included_categories += len(categories.split(", "))
            n_included_patients_with_category += 1
category_of_included_patients = {k : v for k, v in category_of_included_patients.items() if v > 0}
print(f"included {n_included_categories} categories from {n_included_patients_with_category} patients")

In [ ]:
potentially_includable_patients_per_category = {percentage_category_count[c]*df_data_ED.shape[0] for c in category_of_included_patients}

In [ ]:
potentially_includable_patients = round(sum(potentially_includable_patients_per_category))

In [ ]:
df_data_noninclusion_file = "noninclusion.xlsx"
df_data_noninclusion = pd.read_excel(df_data_noninclusion_file, index_col=0, header=None).T
df_data_noninclusion = df_data_noninclusion[['case number *', 'reason for non-inclusion *', 'date of presentation *', 'inclusion criterium AF *', 'inclusion criterium CCS *', 'cohort *']]
df_data_noninclusion

In [ ]:
reasons_for_non_inclusion = [{"label": reason, "id": reason, "n": df_data_noninclusion.loc[df_data_noninclusion["reason for non-inclusion *"]==reason, :].shape[0], "parent": "patients screened in ED"} for reason in df_data_noninclusion["reason for non-inclusion *"].unique()]

In [ ]:
df_data_personal_file = "personal.xlsx"
df_data_personal = pd.read_excel(df_data_personal_file, index_col=0, header=None, nrows=6).T
df_data_excluded_from_personal = df_data_personal.loc[df_data_personal["exclusion *"]=="yes", ["pseudonym", "reason for exclusion"]]
df_data_excluded_from_personal

In [ ]:
reasons_for_exclusion_from_personal = [{"label": reason, "id": reason, "n": df_data_unclean.loc[(df_data_unclean["pseudonym"].isin(df_data_excluded_from_personal.loc[df_data_excluded_from_personal["reason for exclusion"]==reason, "pseudonym"])), :].shape[0], "parent": "patients screened in DZHI" if cohort=="DZHI" else "patients screened in ED"} for reason in df_data_excluded_from_personal["reason for exclusion"].unique() for cohort in cohorts if df_data_unclean.loc[(df_data_unclean["pseudonym"].isin(df_data_excluded_from_personal.loc[df_data_excluded_from_personal["reason for exclusion"]==reason, "pseudonym"])) & (df_data_unclean["cohort"]==cohort), ].shape[0] > 0]

In [ ]:
df_data_excluded_from_data_file = df_data_unclean.loc[ (
    ((df_data_unclean["exclusion due to exclusion criteria"] == "yes") |
     (df_data_unclean["entry done"] == "no"))
    & ~df_data_unclean["pseudonym"].isin(df_data_excluded_from_personal["pseudonym"])
), :]
reasons_for_exclusion_from_data = [{"label": reason, "id": reason, "n": df_data_excluded_from_data_file.loc[df_data_excluded_from_data_file["reason for exclusion"]==reason, :].shape[0], "parent": "patients included"} for reason in df_data_excluded_from_data_file["reason for exclusion"].unique() if df_data_excluded_from_data_file.loc[df_data_excluded_from_data_file["reason for exclusion"]==reason, :].shape[0] > 0]

In [ ]:
n_patients_in_ED = df_data_ED.shape[0]
n_patients_in_DZHI = df_data_DZHI.shape[0]
n_patients_in_ED_with_matching_initial_diagnosis = potentially_includable_patients
n_patients_screened_ED = df_data_unclean.loc[df_data_unclean["cohort"]=="ED", :].shape[0]+df_data_noninclusion.loc[df_data_noninclusion["cohort *"]=="ED", :].shape[0]
n_patients_screened_DZHI = df_data_unclean.loc[df_data_unclean["cohort"]=="DZHI", :].shape[0] + df_data_noninclusion.loc[df_data_noninclusion["cohort *"]=="DZHI", :].shape[0] 
n_patients_included = df_data_unclean.loc[~df_data_unclean["pseudonym"].isin(df_data_excluded_from_personal["pseudonym"]), :].shape[0] 
n_patients_excluded_from_data_file = df_data_unclean.loc[ (
    ((df_data_unclean["exclusion due to exclusion criteria"] == "yes") |
     (df_data_unclean["entry done"] == "no"))
    & ~df_data_unclean["pseudonym"].isin(df_data_excluded_from_personal["pseudonym"]) 
), :].shape[0] 


In [ ]:
patient_flow_data = [
    {"id": "patients in ED", "label": "patients in ED", "n": n_patients_in_ED, "parent": None},
    {"id": "patients in DZHI", "label": "patients in DZHI", "n": n_patients_in_DZHI, "parent": None},
    {"id": "patients in ED with matching initial diagnosis", "label": "patients in ED with matching initial diagnosis", "n": n_patients_in_ED_with_matching_initial_diagnosis, "parent": "patients in ED"},
    {"id": "patients screened in ED", "label": "patients screened in ED", "n": n_patients_screened_ED, "parent": "patients in ED with matching initial diagnosis"},
    {"id": "patients screened in DZHI", "label": "patients screened in DZHI", "n": n_patients_screened_DZHI, "parent": "patients in DZHI"}, 
    {"id": "patients included", "label": "patients included", "n": n_patients_included, "parent": "patients screened in ED"},
    {"id": "patients included", "label": "patients included", "n": n_patients_included, "parent": "patients screened in DZHI"}, 
    {"id": "patients with full data", "label": "patients with full data", "n": df_data.shape[0], "parent": "patients included"},
]

for reason_dict in reasons_for_non_inclusion:
    patient_flow_data.append(reason_dict)
for reason_dict in reasons_for_exclusion_from_personal:
    patient_flow_data.append(reason_dict)
for reason_dict in reasons_for_exclusion_from_data:
    patient_flow_data.append(reason_dict)

In [ ]:
df_flow_data = pd.DataFrame(patient_flow_data)

In [ ]:
df_flow_data.to_csv("patient_flow_data_25-09-19.csv")